### Hugging Face Accelerate Demo

**Note**: Before running this demo, please make sure that you have `wandb.ai` free account. 

Let us install `accelerate`.

In [1]:
!pip install accelerate

     |████████████████████████████████| 58 kB 669 kB/s eta 0:00:01


**Import** the required modules.

In [3]:
import torch
import torchvision
import wandb
import datetime
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ui import progress_bar

# This is a demo of the PyTorch Accelerate API.
from accelerate import Accelerator

**`wandb`** initialization. See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.


In [4]:
wandb.login()
config = {
  "learning_rate": 0.1,
  "epochs": 100,
  "batch_size": 128,
  "dataset": "cifar10"
}
run = wandb.init(project="accelerate-project", entity="upeee", config=config)

wandb: Currently logged in as: rowel (use `wandb login --relogin` to force relogin)
2022-03-11 10:00:23.760100: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-11 10:00:23.760140: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Build the model

Use a ResNet18 from `torchvision`. See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [8]:
# Shows the code to be replaced with the Accelerate API.
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accelerator = Accelerator()

model = torchvision.models.resnet18(pretrained=False, progress=True)

model.fc = torch.nn.Linear(model.fc.in_features, 10) 

# Replace the model with the Accelerate API.
#model.to(device)

# watch model gradients during training
wandb.watch(model)

[]

### Loss function, Optimizer, Scheduler and DataLoader

See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.


In [9]:
loss = torch.nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=wandb.config.learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=wandb.config.epochs)

x_train = datasets.CIFAR10(root='./data', train=True, 
                           download=True, 
                           transform=transforms.ToTensor())
x_test = datasets.CIFAR10(root='./data',
                          train=False, 
                          download=True, 
                          transform=transforms.ToTensor())
train_loader = DataLoader(x_train, 
                          batch_size=wandb.config.batch_size, 
                          shuffle=True, 
                          num_workers=2)
test_loader = DataLoader(x_test, 
                         batch_size=wandb.config.batch_size, 
                         shuffle=False, 
                         num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### Visulaizing sample data from test split


See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.


Note the last line that uses Accelerate API to wrap the model, optimizer, data loaders and scheduler.

In [10]:

label_human = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

table_test = wandb.Table(columns=['Image', "Ground Truth", "Initial Pred Label",])

image, label = iter(test_loader).next()
model.eval()
with torch.no_grad():
  pred = torch.argmax(model(image), dim=1).cpu().numpy()
  # Replace the model with the Accelerate API.
  #pred = torch.argmax(model(image.to(device)), dim=1).cpu().numpy()

for i in range(8):
  table_test.add_data(wandb.Image(image[i]),
                      label_human[label[i]], 
                      label_human[pred[i]])
  print(label_human[label[i]], "vs. ",  label_human[pred[i]])

# Accelerate API
model, optimizer, scheduler, train_loader, test_loader = accelerator.prepare(model,
                                                                             optimizer,
                                                                             scheduler, 
                                                                             train_loader, 
                                                                             test_loader)

cat vs.  deer
ship vs.  deer
ship vs.  deer
airplane vs.  deer
frog vs.  deer
frog vs.  deer
automobile vs.  deer
frog vs.  deer


### The train loop

Using Accelerate, we do not need to transfer the model to the `device`.

See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [11]:
def train(epoch):
  model.train()
  train_loss = 0
  correct = 0
  train_samples = 0

  # sample a batch. compute loss and backpropagate
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    # Replaced by the Accelerate API.
    #target = target.to(device)
    #output = model(data.to(device))
    
    output = model(data)
    loss_value = loss(output, target)

    # Replaced by the Accelerate API.
    #loss_value.backward()
    accelerator.backward(loss_value)

    optimizer.step()
    scheduler.step(epoch)
    train_loss += loss_value.item()
    train_samples += len(data)
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()
    if batch_idx % 10 == 0:
      accuracy = 100. * correct / len(train_loader.dataset)
      progress_bar(batch_idx,
                   len(train_loader),
                   'Train Epoch: {}, Loss: {:.6f}, Acc: {:.2f}%'.format(epoch+1, 
                   train_loss/train_samples, accuracy))
  
  train_loss /= len(train_loader.dataset)
  accuracy = 100. * correct / len(train_loader.dataset)

  return accuracy, train_loss

### The validation loop

After every epoch, we will run the validation loop for the model. Again, no need to transfer the data to the `device`.

See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [12]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:

      # Replaced by the Accelerate API.
      #output = model(data.to(device))   
      #target = target.to(device)

      output = model(data)
      test_loss += loss(output, target).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print('\nTest Loss: {:.4f}, Acc: {:.2f}%\n'.format(test_loss, accuracy))

  return accuracy, test_loss

### `wandb` plots

Finally, we will use `wandb` to visualize the training progress. 
See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [13]:
run.display(height=1000)

start_time = datetime.datetime.now()
best_acc = 0
for epoch in range(wandb.config["epochs"]):
    train_acc, train_loss = train(epoch)
    test_acc, test_loss = test()
    if test_acc > best_acc:
        wandb.run.summary["Best accuracy"] = test_acc
        best_acc = test_acc
        accelerator.save(model, "resnet18_best_acc.pth")
    wandb.log({
        "Train accuracy": train_acc,
        "Test accuracy": test_acc,
        "Train loss": train_loss,
        "Test loss": test_loss,
        "Learning rate": optimizer.param_groups[0]['lr']
    })

elapsed_time = datetime.datetime.now() - start_time
print("Elapsed time: %s" % elapsed_time)
wandb.run.summary["Elapsed train time"] = str(elapsed_time)

model.eval()
with torch.no_grad():
  pred = torch.argmax(model(image), dim=1).cpu().numpy()
  # Replace the model with the Accelerate API.
  #pred = torch.argmax(model(image.to(device)), dim=1).cpu().numpy()

final_pred = []
for i in range(8):
    final_pred.append(label_human[pred[i]])
    print(label_human[label[i]], "vs. ",  final_pred[i])

table_test.add_column(name="Final Pred Label", data=final_pred)

wandb.log({"Test data": table_test})

wandb.finish()

/home/rowel/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


/home/rowel/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


 [=============================>]  Step: 163ms | Tot: 6s656ms | Train Epoch: 1, Loss: 0.012896, Acc: 43.67 391/391 

Test Loss: 0.0180, Acc: 38.62%

 [=============================>]  Step: 165ms | Tot: 6s614ms | Train Epoch: 2, Loss: 0.008939, Acc: 59.61 391/391 

Test Loss: 0.0100, Acc: 55.81%

 [=============================>]  Step: 162ms | Tot: 6s727ms | Train Epoch: 3, Loss: 0.007248, Acc: 67.52 391/391 

Test Loss: 0.0106, Acc: 54.48%

 [=============================>]  Step: 166ms | Tot: 6s699ms | Train Epoch: 4, Loss: 0.006077, Acc: 72.66 391/391 

Test Loss: 0.0141, Acc: 48.35%

 [=============================>]  Step: 165ms | Tot: 6s738ms | Train Epoch: 5, Loss: 0.005158, Acc: 76.94 391/391 

Test Loss: 0.0085, Acc: 64.41%

 [=============================>]  Step: 178ms | Tot: 6s740ms | Train Epoch: 6, Loss: 0.004339, Acc: 80.58 391/391 

Test Loss: 0.0078, Acc: 68.59%

 [=============================>]  Step: 161ms | Tot: 6s568ms | Train Epoch: 7, Loss: 0.003588, Acc: 83.85

 [=============================>]  Step: 165ms | Tot: 6s688ms | Train Epoch: 55, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0172, Acc: 75.48%

 [=============================>]  Step: 163ms | Tot: 6s607ms | Train Epoch: 56, Loss: 0.000001, Acc: 100.00 391/391 

Test Loss: 0.0168, Acc: 75.72%

 [=============================>]  Step: 164ms | Tot: 6s642ms | Train Epoch: 57, Loss: 0.000001, Acc: 100.00 391/391 

Test Loss: 0.0170, Acc: 75.68%

 [=============================>]  Step: 164ms | Tot: 6s700ms | Train Epoch: 58, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0166, Acc: 75.66%

 [=============================>]  Step: 165ms | Tot: 6s703ms | Train Epoch: 59, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0167, Acc: 75.66%

 [=============================>]  Step: 163ms | Tot: 6s673ms | Train Epoch: 60, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0169, Acc: 75.89%

 [=============================>]  Step: 171ms | Tot: 6s748ms | Train Epoch: 61, Loss: 0.00000

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

### Load the best performing model

In the following code, we load the best performing model. The model is saved in `./resnet18_best_acc.pth`. The average accuracy of the model is the same as the one in the previous section.

In [14]:
model = torch.load("resnet18_best_acc.pth")
# Using Accelerator API
model = accelerator.prepare(model)
accuracy, _ = test()
print("Best accuracy: %.2f" % accuracy)


Test Loss: 0.0168, Acc: 75.90%

Best accuracy: 75.90
